In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from privacypacking.utils.utils import load_logs, global_metrics
import pandas as pd
from experiments.ray.analysis import load_ray_experiment, load_latest_ray_experiment, load_latest_scheduling_results, load_latest_scheduling_results, load_latest_ray_experiment
import plotly.express as px
from privacypacking.budget.curves import  LaplaceCurve, GaussianCurve, SubsampledGaussianCurve
from privacypacking.budget import Budget, Task, Block
from privacypacking.schedulers.metrics import OverflowRelevance, FlatRelevance
from privacypacking.budget.block_selection import RandomBlocks
from privacypacking.utils.plot import plot_budgets
import yaml
from pathlib import Path
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from experiments.ray.analysis import get_percentiles

In [4]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-14-39"))

In [6]:
rdf.head()

,scheduler,solver,scheduler_n,scheduler_metric,block_selecting_policy,frequency_file,n_allocated_tasks,total_tasks,realized_profit,n_tasks,...,date,timestamp,time_total_s,pid,hostname,node_ip,config,time_since_restore,timesteps_since_restore,iterations_since_restore
0,time_based_budget_unlocking,gurobi,1,DominantShares,LatestBlocksFirst,frequencies.yaml,783,1701,783.0,1701,...,2021-12-07_09-15-21,1638886521,7.411311,667399,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",7.411311,0,1
1,time_based_budget_unlocking,gurobi,1,DominantShares,LatestBlocksFirst,frequencies.yaml,681,1701,681.0,1701,...,2021-12-07_09-15-09,1638886509,25.917010,667412,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",25.917010,0,1
2,time_based_budget_unlocking,gurobi,1,VectorizedBatchOverflowRelevance,LatestBlocksFirst,frequencies.yaml,715,1701,715.0,1701,...,2021-12-07_09-15-39,1638886539,57.999942,667392,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",57.999942,0,1
3,time_based_budget_unlocking,gurobi,1,Fcfs,LatestBlocksFirst,frequencies.yaml,683,1701,683.0,1701,...,2021-12-07_09-15-12,1638886512,7.293786,667405,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",7.293786,0,1
4,time_based_budget_unlocking,gurobi,1,DynamicFlatRelevance,LatestBlocksFirst,frequencies.yaml,715,1701,715.0,1701,...,2021-12-07_09-15-53,1638886553,70.765050,667407,ds-09,128.59.23.59,"{'epsilon': 10, 'delta': 1e-05, 'global_seed':...",70.765050,0,1


In [23]:
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, no profits, mixed curves. Fully unlocked. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [22]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-06_20-25-51"))
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, with profits, mixed curves. Fully unlocked. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [21]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-22-04"))
px.line(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="T",
    y="n_allocated_tasks",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, no profits, mixed curves. Unlocking with lifetime 5. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)



In [20]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_09-39-00"))
px.line(
    rdf.query("scheduler_metric != 'FlatRelevance'").sort_values(["T", "scheduler_metric"]),
    x="T",
    y="realized_profit",
    color="scheduler_metric",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    title="20 blocks, with profits, mixed curves. Unlocking with lifetime 5. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [30]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_10-32-11"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [31]:
# Tuning
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-28-33"))
rdf.columns
px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [33]:
# Temp tuning with profits
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_14-55-57"))
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-04-36"))

px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [36]:
# rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-08-13"))

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-18-48"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [38]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_15-52-12"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline.  Poly ratio. Changing hyperparameters",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [4]:

rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_18-34-39"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="realized_profit",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, with profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [5]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_19-32-46"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [6]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_20-34-49"))


px.line(
    rdf.sort_values(["temperature", "metric_recomputation_period"]),
    x="temperature",
    y="n_allocated_tasks",
    log_x=True,
    color="metric_recomputation_period",
    # log_x=True,
    width=800,
    height=600,
    range_y=[0,1200],
    facet_col="normalize_by",
    facet_col_wrap=1,
    title="20 blocks, no profits, mixed curves. Offline. Softmax with subtracting the max.",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)

In [14]:
# On privatekube now!!


rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-29-50"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-42-40"))])
px.bar(
    rdf.sort_values(["T", "scheduler_metric"]),
    # x="T",
    x="scheduler_metric",    
    y="realized_profit",
    # color="scheduler_metric",
    # log_x=True,
    width=1000,
    height=600,
    range_y=[0,6_000],
    title="10 initial blocks + 20 online blocks, with profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)


In [15]:
rdf = load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-40-36"))
rdf = pd.concat([rdf, load_ray_experiment(Path("/home/pierre/privacypacking/logs/ray/run_and_report_2021-12-07_21-43-03"))])
px.bar(
    rdf.sort_values(["T", "scheduler_metric"]),
    x="scheduler_metric",
    y="n_allocated_tasks",
    # log_x=True,
    width=1000,
    height=600,
    range_y=[0,10_000],
    title="10 initial blocks + 20 online blocks, NO profits, PrivateKube. Unlocking with lifetime 10. Temperature = 1",
    # title="Number of allocated tasks depending on the scheduling step size<br><sup>Online mixed curves, 20 blocks, no initial blocks, 100 tasks per block on average, lifetime = 5 blocks</sup>"
)